# Обучение FastText

В бейзлайне предполагается использовать векторы из библиотеки FastText, которые в данном ноутбуке обучатся на домен чеков

In [2]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np

Выгрузим все чеки из размеченного датасета для обучения, а также из тестового датасета

In [3]:
train_df = pd.read_csv("data/train_supervised_dataset.csv").fillna("")
uns_train_df = pd.read_csv("data/train_unsupervised_dataset.csv").fillna("")
test_df = pd.read_csv("data/test_dataset.csv")
names = pd.concat((train_df[["name"]], test_df, uns_train_df[["name"]])).reset_index(drop=True)
names

,name,id
0,Petmax Бантик леопард с красн розой 2шт,NaN
1,87191 Бусы для елки шарики_87191,NaN
2,Футболка Piazza Italia WR011446881,NaN
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN
4,Одежда (вес) 1500,NaN
...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN


In [4]:
train_df['good'].value_counts(1)

               0.03276
брюки          0.01356
пиво           0.01308
вода           0.01232
печенье        0.01164
                ...   
молитвослов    0.00004
медведь        0.00004
супница        0.00004
овчина         0.00004
поддон         0.00004
Name: good, Length: 2820, dtype: float64

In [5]:
train_df['brand'].value_counts(1)

                        0.34020
яшкино                  0.00480
агуша                   0.00364
коровка из кореновки    0.00272
махеевъ                 0.00252
                         ...   
пласти лаб              0.00004
феррогематоген          0.00004
cbr                     0.00004
helcom                  0.00004
smail                   0.00004
Name: brand, Length: 6976, dtype: float64

In [257]:
import re

from cyrtranslit import to_cyrillic, to_latin
lat_to_cyr = str.maketrans("aekmhopctyx", "аекмнорстух")
cyr_to_lat = str.maketrans("аекмнорстух", "aekmhopctyx")
stop_words = set(['г', 'кг', 'шт', 'мл', 'л', 'литр', 'мг', 'гр', 'км', 'мм', 'mm', 'уп'])

def replace_camel_case(s):
    matches = len(re.findall(r'(?<=[a-zа-я])([A-ZА-Я])', s))
    if matches > 2:
        return re.sub(r'(?<=[a-zа-я])([A-ZА-Я])', r' \1', s)
    else:
        return s
    
def split_on_language_change(s):
    s = re.sub(r'(?<=[a-zа-я])(?=[A-ZА-Я0-9])', r' ', s)
    s = re.sub(r'(?<=[A-ZА-Я0-9])(?=[a-zа-я])', r' ', s)
    return s


def insert_space_after_one(s):
    return re.sub(r'(1)(?=[A-Za-zА-Яа-я])', r'\1 ', s)

def replace_zero(s):
    s = re.sub(r'(?<=[A-Za-z])0(?=[A-Za-z])', 'o', s)
    s = re.sub(r'(?<=[А-Яа-я])0(?=[А-Яа-я])', 'о', s)
    return s

def preprocess_text(text):
    
    text = replace_zero(text)
    
    text = re.sub('\d+', '1', text)  # replace numbers to 1
    text = replace_camel_case(text)
    text = re.sub('д/', 'для ', text)
    text = insert_space_after_one(text)
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    words = []
    for w in text.lower().split():
        if len(w) < 2:
            continue
        
        num_eng_chars = len(re.findall(r'[a-z]', w))
        num_ru_chars = len(re.findall(r'[а-я]', w))
        if num_eng_chars and num_ru_chars:
            if num_eng_chars > num_ru_chars:
                w = w.translate(cyr_to_lat)
            else:
                w = w.translate(lat_to_cyr)
        
        if w in stop_words:
            continue
        
        # если нет транзиторов
        w = split_on_language_change(w)
        if ' ' in w:
            words.extend(w.split())
        else:
            words.append(w)

        
    return words

In [258]:
text = train_df['name'].sample().values[0]
text

'1) 001183|Диск сцепления 2123 VALEO 809726'

In [259]:
train_df[train_df['name'] == text]

,id,name,good,brand
8800,8800,1) 001183|Диск сцепления 2123 VALEO 809726,диск,valeo


In [194]:
%%timeit
preprocess_text(text)

31.1 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [260]:
preprocess_text(text)

['диск', 'сцепления', 'valeo']

In [261]:
%%time
names["tokens"] = names["name"].apply(preprocess_text)
names

CPU times: user 48.6 s, sys: 765 ms, total: 49.4 s
Wall time: 49.4 s


,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, красн, розой]"
1,87191 Бусы для елки шарики_87191,NaN,"[бусы, для, елки, шарики_1]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr, 1]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[yi, 1, one, заколка, для, волос, для, девочки]"
4,Одежда (вес) 1500,NaN,"[одежда, вес]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[спонж, для, макияжа, фигурный]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,"[дифф, arom]"
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, см, tropical, bird, запл, д..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, сп, размер, х, 1, ..."


In [266]:
names.tokens.apply(len).describescribe()

count    1.030000e+06
mean     5.061572e+00
std      2.434675e+00
min      0.000000e+00
25%      3.000000e+00
50%      5.000000e+00
75%      6.000000e+00
max      4.400000e+01
Name: tokens, dtype: float64

In [267]:
names[names.tokens.apply(len) == 0]

,name,id,tokens
4411,А м е р и к а н о / с р е д н и й,NaN,[]
26135,4627145110829 100.00 * 1 ШТ=100.00,1135.0,[]
30684,9 мм,NaN,[]
32700,42.361.0F,NaN,[]
38106,№ 01,NaN,[]
...,...,...,...
1000570,А Р C,NaN,[]
1002512,W 868/36 (311),NaN,[]
1011191,!11 60.0% 214мл,NaN,[]
1014311,"0,5 л.",NaN,[]


In [262]:
names.to_csv('data/preprocc_names.csv', index=False)

Обученные векторы положим в файл `fasttext.model` для использования в моделях

In [263]:
%%time
FastText(names["tokens"], vector_size=128, window=5, min_count=5, epochs=20).save("fasttext_models/fasttext_128_preprocc_fitted.model")

CPU times: user 11min 58s, sys: 8.17 s, total: 12min 6s
Wall time: 4min 36s
